In [1]:
!python -V

Python 3.9.21


In [2]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db") 
mlflow.set_experiment("ny-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/1', creation_time=1746683774913, experiment_id='1', last_update_time=1746683774913, lifecycle_stage='active', name='ny-taxi-experiment', tags={}>

In [3]:
#install pyarrow to read parquet file

#!pip install pyarrow

In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error



In [5]:
#!wget -P data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet 

In [6]:
#!wget -P data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet 

In [14]:
def read_dataframe(filename):
    df = pd.read_parquet(filename,engine='pyarrow')

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [16]:


df_train = read_dataframe('./data/green_tripdata_2021-02.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-01.parquet')


In [17]:
df_train['PU_DO'] = df_train['PULocationID'].astype(str) + '_' + df_train['DOLocationID'].astype(str)
df_val['PU_DO'] = df_val['PULocationID'].astype(str) + '_' + df_val['DOLocationID'].astype(str)


In [18]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)


In [19]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values


In [20]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred)


50.01983345993515

In [21]:
#Upload model to the model folder dv is data set vectoried and ltr is model
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)


In [24]:
with mlflow.start_run():
    mlflow.set_tag("developer", "Shakti")
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-02.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-01.parque")
    alpha = 0.1
    mlflow.log_param("alpha",alpha)



    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)

    rmse= mean_squared_error(y_val, y_pred)
    mlflow.log_param("rmse",rmse)


In [ ]:
import xgboost as xgb

#hyperopt used to find best parameters
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

def objective(parsms):
    with mlflow.start_run():
         mlflow.set_tag("model", "xgboost")
         mlflow.log_params(params)
         booster = xgb.train(
              params=params,
              dtrain=train,
              num_boost_round=1000,
              evals=[(valid, "validation")],
              early_stopping_rounds=50
         )
